In [ ]:
!pip install --user tensorflow-hub

In [ ]:
!pip install --user transformers

In [ ]:

!unzip c15a067b44a328c7d5a03c79070b7865f444d1e3.zip

In [ ]:
question_answer = __import__('0-qa').question_answer

with open('ZendeskArticles/PeerLearningDays.md') as f:
    reference = f.read()

print(question_answer('when is PLDs?', reference))

In [ ]:
!chmod u+x 2-qa.py

In [ ]:
!./2-qa.py

In [ ]:
import shlex
import tensorflow as tf
import tensorflow_hub as hub
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
model = hub.load("https://www.kaggle.com/models/seesee/bert/frameworks/TensorFlow2/variations/uncased-tf2-qa/versions/1")

# Adjust the file path based on where you uploaded the document in Colab
with open('/content/ZendeskArticles/PeerLearningDays.md') as f:
    reference = f.read()

class CustomCommand:
    prompt = 'Q: '

    def __init__(self):
        self.exit_flag = False

    def run_console(self):
        print("Type 'exit' to end the conversation.")
        while not self.exit_flag:
            user_input = input(self.prompt)
            self.handle_input(user_input)

    def handle_input(self, user_input):
        if user_input.lower() in ['quit', 'exit', 'goodbye', 'bye']:
            print('A: Goodbye')
            self.exit_flag = True
        else:
            answer = question_answer(user_input, reference)
            print(f'A: {answer}')

def question_answer(question, reference):
    """using `[1:]` will enforce an answer.
    `outputs[0][0][0]` is the ignored '[CLS]' token logit"""
    question_tokens = tokenizer.tokenize(question)
    paragraph_tokens = tokenizer.tokenize(reference)
    tokens = ['[CLS]'] + question_tokens + \
        ['[SEP]'] + paragraph_tokens + ['[SEP]']
    input_word_ids = tokenizer.convert_tokens_to_ids(tokens)
    input_mask = [1] * len(input_word_ids)
    input_type_ids = [0] * (1 + len(question_tokens) + 1) + \
        [1] * (len(paragraph_tokens) + 1)
    input_word_ids, input_mask, input_type_ids = map(lambda t: tf.expand_dims(
        tf.convert_to_tensor(t, dtype=tf.int32), 0), (input_word_ids, input_mask, input_type_ids))
    outputs = model([input_word_ids, input_mask, input_type_ids])
    short_start = tf.argmax(outputs[0][0][1:]) + 1
    short_end = tf.argmax(outputs[1][0][1:]) + 1
    answer_tokens = tokens[short_start: short_end + 1]
    answer = tokenizer.convert_tokens_to_string(answer_tokens)
    return answer

# Create an instance of the console and run it



In [ ]:
console = CustomCommand()
console.run_console()

In [ ]:
!pip install -U sentence-transformers

In [ ]:
!pip install -U scikit-learn

In [ ]:
import os
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

corpus = []
for filename in os.listdir("ZendeskArticles"):
    if filename.endswith(".md"):
        file_path = os.path.join("ZendeskArticles", filename)
        with open(file_path, 'r') as md_file:
            corpus.append(md_file.read() + '\n')

model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings_sen = model.encode("When are PLDs?")

similarities = [cosine_similarity([embeddings_sen], [model.encode(doc)]) for doc in corpus]
most_similar_idx = np.argmax(similarities)
most_similar_doc = corpus[most_similar_idx]

print("Most similar document:")
print(most_similar_doc)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Most similar document:
PLD Overview
Peer Learning Days (PLDs) are a time for you and your peers to ensure that each of you understands the concepts you've encountered in your projects, as well as a time for everyone to collectively grow in technical, professional, and soft skills. During PLD, you will collaboratively review prior projects with a group of cohort peers.
PLD Basics
PLDs are mandatory on-site days from 9:00 AM to 3:00 PM. If you cannot be present or on time, you must use a PTO. 
No laptops, tablets, or screens are allowed until all tasks have been whiteboarded and understood by the entirety of your group. This time is for whiteboarding, dialogue, and active peer collaboration. After this, you may return to computers with each other to pair or group program. 
Peer Learning Days are not about sharing solutions. This doesn't empower peers with the ability to solve problems themselves! Peer learning is when you share your thought process, whether through conversation, whiteboa

In [ ]:
import os
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np


def semantic_search(corpus_path, sentence):
    corpus = []
    for filename in os.listdir(corpus_path):
        if filename.endswith(".md"):
            file_path = os.path.join(corpus_path, filename)
            with open(file_path, 'r') as md_file:
                corpus.append(md_file.read() + '\n')

    model = SentenceTransformer("all-MiniLM-L6-v2")
    embeddings_sen = model.encode("When are PLDs?")

    similarities = [cosine_similarity(
        [embeddings_sen], [model.encode(doc)]) for doc in corpus]
    most_similar_idx = np.argmax(similarities)
    most_similar_doc = corpus[most_similar_idx]
    return most_similar_doc


In [ ]:
semantic_search = __import__('3-semantic_search').semantic_search

print(semantic_search('ZendeskArticles', 'When are PLDs?'))

PLD Overview
Peer Learning Days (PLDs) are a time for you and your peers to ensure that each of you understands the concepts you've encountered in your projects, as well as a time for everyone to collectively grow in technical, professional, and soft skills. During PLD, you will collaboratively review prior projects with a group of cohort peers.
PLD Basics
PLDs are mandatory on-site days from 9:00 AM to 3:00 PM. If you cannot be present or on time, you must use a PTO. 
No laptops, tablets, or screens are allowed until all tasks have been whiteboarded and understood by the entirety of your group. This time is for whiteboarding, dialogue, and active peer collaboration. After this, you may return to computers with each other to pair or group program. 
Peer Learning Days are not about sharing solutions. This doesn't empower peers with the ability to solve problems themselves! Peer learning is when you share your thought process, whether through conversation, whiteboarding, debugging, or li

In [ ]:
#!/usr/bin/env python3

answer_loop = __import__('2-qa').answer_loop

with open('ZendeskArticles/PeerLearningDays.md') as f:
    reference = f.read()

answer_loop(reference)


Q: What are Mock Interviews?
A: Sorry, I do not understand your question.
Q: pld
A: peer learning days
Q: when pld
A: 9 : 00 am to 3 : 00 pm
Q: bye
A: Goodbye


SystemExit: 

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
#!/usr/bin/env python3

question_answer = __import__('4-qa').question_answer

question_answer('ZendeskArticles')

TypeError: question_answer() missing 1 required positional argument: 'reference'

In [ ]:
#!/usr/bin/python3
""" console """

import sys
import tensorflow as tf
import tensorflow_hub as hub
from transformers import BertTokenizer
import os
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
tokenizer = BertTokenizer.from_pretrained(
    'bert-large-uncased-whole-word-masking-finetuned-squad')
model = hub.load(
    "https://www.kaggle.com/models/seesee/bert/frameworks/" +
    "TensorFlow2/variations/uncased-tf2-qa/versions/1")


def question_answer(question, reference):
    """using `[1:]` will enforce an answer.
    `outputs[0][0][0]` is the ignored '[CLS]' token logit"""
    question_tokens = tokenizer.tokenize(question)
    paragraph_tokens = tokenizer.tokenize(reference)
    tokens = ['[CLS]'] + question_tokens + \
        ['[SEP]'] + paragraph_tokens + ['[SEP]']
    input_word_ids = tokenizer.convert_tokens_to_ids(tokens)
    input_mask = [1] * len(input_word_ids)
    input_type_ids = [0] * (1 + len(question_tokens) + 1) + \
        [1] * (len(paragraph_tokens) + 1)
    input_word_ids, input_mask, input_type_ids = map(lambda t: tf.expand_dims(
        tf.convert_to_tensor(t, dtype=tf.int32), 0), (input_word_ids, input_mask, input_type_ids))
    outputs = model([input_word_ids, input_mask, input_type_ids])
    short_start = tf.argmax(outputs[0][0][1:]) + 1
    short_end = tf.argmax(outputs[1][0][1:]) + 1
    answer_tokens = tokens[short_start: short_end + 1]
    answer = tokenizer.convert_tokens_to_string(answer_tokens)
    return answer


def semantic_search(corpus_path, sentence):
    corpus = []
    for filename in os.listdir(corpus_path):
        if filename.endswith(".md"):
            file_path = os.path.join(corpus_path, filename)
            with open(file_path, 'r') as md_file:
                corpus.append(md_file.read() + '\n')

    model = SentenceTransformer("all-MiniLM-L6-v2")
    embeddings_sen = model.encode(sentence)

    similarities = [cosine_similarity(
        [embeddings_sen], [model.encode(doc)]) for doc in corpus]
    most_similar_idx = np.argmax(similarities)
    most_similar_doc = corpus[most_similar_idx]
    return most_similar_doc


def answer_loop(coprus_path):
    while True:
        try:
            sentence = input("Q: ")
            sentence = sentence.lower()
            if sentence in ['exit', 'quit', 'goodbye', 'bye']:
                print("A: Goodbye")
                sys.exit()
            else:
                reference = semantic_search(coprus_path, sentence)
                if question_answer(sentence, reference) == None or question_answer(sentence, reference) == "":
                    print("A: Sorry, I do not understand your question.")
                else:
                    print("A: {}".format(question_answer(sentence, reference)))
        except (KeyboardInterrupt, EOFError):
            print("A: Goodbye")
            sys.exit()


if __name__ == "__main__":
    answer_loop('ZendeskArticles')

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

Q: When are PLDs?
A: on - site days from 9 : 00 am to 3 : 00 pm
Q: What are Mock Interviews?
A: each student in a pair will receive a different script to interview their partner
Q: What does PLD stand for?
A: peer learning days
Q: bye
A: Goodbye


SystemExit: 

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
!pip install Flask tensorflow-serving-api

In [ ]:
!tensorflow_model_server --port=8500 --rest_api_port=8501 --model_name=qa_model --model_base_path=/path/to/your/exported_model


/bin/bash: line 1: tensorflow_model_server: command not found


In [ ]:
!pip install tensorflow-serving-api


In [ ]:
!tensorflow_model_server --port=8500 --rest_api_port=8501 --model_name=qa_model --model_base_path=/path/to/your/exported_model


/bin/bash: line 1: tensorflow_model_server: command not found


In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
!tensorflow_model_server --port=8500 --rest_api_port=8501 --model_name=qa_model --model_base_path=/path/to/your/exported_model

/bin/bash: line 1: tensorflow_model_server: command not found
